# HW 1 - Data Platform IDC
ID1: 308564293

ID2: XXX

## Step 1: Setup Cassandra Cluster locally -

> Code is in markdown as we don't want to run this from the notebook
> The setup is oriented for mac env

Make sure Docker Desktop is installed. If not, download and install Docker from the official website [Get Docker | Docker Docs](https://docs.docker.com/get-docker/).

Download docker image for cassandra
```bash
docker pull cassandra:latest
```

Run container locally open for conncetion from the notebook
```bash
docker run --volume=/var/lib/cassandra --restart=no -p 127.0.0.1:9042:9042 -p 127.0.0.1:9160:9160 --name hw-cass -d cassandra:latest
```

Connect to run `cqlsh` locally
```bash
docker exec -it hw-cass cqlsh
```

## Step 2: setup python environment

install python
```bash
brew install python
```

install required libraries

cassandra driver
```bash
pip3 install cassandra-driver
```

## Step 3: Chossing the dataset

##### Data set content - [Dataset link.](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data)
The Book-Crossing dataset has 3 files and is given in .csv form.


`Users` Has 3 colums:
* User-ID (int): Anonymized user identifier mapped to integers.
* Location (text): Demographic information about the user's location.
* Age (int): Age of the user. May contain NULL-values if not available.

Rows:  279K, each identifing a user.

`Books` Has 8 columns:
* SBN (text): Unique identifier for each book.
* Book-Title (text): Title of the book.
* Book-Author (text): Author of the book. In case of multiple authors, only the first one is provided.
* Year-Of-Publication (int): The year when the book was published.
* Publisher (text): The publisher of the book.
* Image-URL-S, Image-URL-M, Image-URL-L (text): URLs linking to cover images in small, medium, and large sizes.

Rows: 271K, that identify books.

`Ratings` Has 3 columns:
* User-ID (int): Anonymized user identifier corresponding to Users table.
* ISBN (text): Unique identifier corresponding to Books table.
* Book-Rating (int): User's rating for a particular book. Explicit ratings are on a scale from 1-10, while implicit ratings are expressed as 0.

Rows: 1.14M, each representing an interaction where a user rated a specific (single) book.


#### Dataset Selection
* The dataset was selected for it's very large size (14 columsn and over 1.5M rows) that will allow us to explore Cassandra's strengths with large datasets.
* The dataset is also realistic, reflecting a real-life scenario of different data soruces and systems that are interconnected. 
* Data is relatively well organized and clean.

**In this project:** The data set is stored in the `Data` folder - where the names of the files are corresponding to the explnation above.

## Step 4: Cassendra Database design
Our dataset will consist of three Cassandra tables that correspond to the different entities in the DB: `Ratings`, `Books` and `Users`.

Each table has a different purpose, information, keys and columns. 
Further, We can assume each entity has originated form a differnet data collection method, so it makes senese to keep them separated in three tables.

##### `book_ratings`
* **Structure**: User-ID: int, ISBN: text, Book-Rating: int
* **Primary key**: (User-ID, ISBN)
* **Partition Key**: User-ID
* **Clustering Column**: ISBN

Since we need a pair of user ID and ISBN to identify a transaction, they are both included in the primary key.
    
The partition key is selected to be User-ID as it has high cardinality (even spread of data).

##### `books`
* **Structure**: ISBN: text, Book-Title: text, Book-Author: text, Year-Of-Publication: int, Publisher: text, Image-URL-S: text, Image-URL-M: text, Image-URL-L: text
* **Primary key**: (ISBN, Book-Author, Publisher)
* **Partition Key**: ISBN
* **Clustering Column**: Book-Author, Publisher

ISBN is a unique identifier per book so it's a good partition key. Clustering keys are seleted to be publisher and author which are expected to have multiple entries.

##### `users`
* **Structure**: User-ID: int, Location: text, Age: int
* **Primary key**: (User-ID, Location, Age)
* **Partition Key**: User-ID
* **Clustering Column**: Location, Age

User-ID is a unique identifier per user so it's a good partition key. Clustering keys are seleted to be Location and Age which are expected aggregations.



## Step 5: Setup keyspace and tables

First - We would like to load all the libraries need for ingestion and working with the cassandra DB

In [88]:
# Cassandra-driver
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, BatchStatement

# Data
import csv
import concurrent.futures

Connect to our cassandra instance.

In [89]:
# Connect Cassandra-Driver to the Cluster running on the Docker:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

Create out key space `books`.
We will use `SimpleStrategy` and `replication_factor` = `1` as they serve us well for the purpose of this excsrsize - as we are not looking for any HA or significant scale

In [90]:
session.execute("CREATE KEYSPACE IF NOT EXISTS books WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")
session.execute("USE books");

Make sure keyspace created

In [91]:
session.execute("DESCRIBE books;").one()

Row(keyspace_name='books', type='keyspace', name='books', create_statement="CREATE KEYSPACE books WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}  AND durable_writes = true;")

Use the keyspace

In [92]:
session.execute("USE books");

We will set up a helper function to assist in executing an ingestion of a single row.

In [107]:
session.execute("""
    CREATE TABLE IF NOT EXISTS books (
        ISBN text,
        Book_Title text,
        Book_Author text,
        Year_Of_Publication int,
        Publisher text,
        Image_URL_S text,
        Image_URL_M text,
        Image_URL_L text,
        PRIMARY KEY (ISBN, Book_Author, Publisher)
    )
""")

In [106]:
session.execute("""
    CREATE TABLE IF NOT EXISTS book_ratings (
        User_ID text,
        ISBN text,
        Book_Rating int,
        PRIMARY KEY(User_ID, ISBN)
    )
""")

In [108]:
session.execute("""
    CREATE TABLE IF NOT EXISTS users (
        User_ID text,
        Location text,
        Age float,
        PRIMARY KEY (User_ID, Location, Age)
    )
""")

## Step 6: Data ingestion

*Data ingestion function*
`load_data` function reads data from a CSV file, splits it into batches, and inserts the data into a table using concurrency.

Batches and concurrency are important for improving the performance and efficiency of inserting large amounts of data into a database table.

When inserting data into a database table, it is possible to insert row by row, or to insert multiple rows at once as a batch. Inserting data in a batch greatly improves the performance of inserting data into the database, as it reduces the number of round trips to the database and can ensure that the data is consistent. This is more efficient than inserting one row at a time, which can be slow and can lead to unnecessary overhead.

Concurrency is important because it allows multiple threads to be used when inserting data into a database table. This improves performance by allowing multiple inserts to happen simultaneously, which can greatly increase the speed of inserting large amounts of data. Without concurrency, each insert operation would have to wait for the previous insert to complete, leading to a slower overall process.

In summary, using batches and concurrency can greatly improve the performance and efficiency of inserting large amounts of data into a database table, resulting in faster insert times and better use of system resources.



In [96]:
def load_data(csv_file, insert_query, batch_size=100, concurrency=20):        
    
    def insert_data(rows):
        batch = BatchStatement()
        for row in rows:
            age = float(row[2]) if row[2] else None
            batch.add(SimpleStatement(insert_query), (row[0], row[1], age))
        session.execute(batch)
    
    with open(csv_file, "r") as f:
        next(f)  # Skip the header row.
        reader = list(csv.reader(f))

    # split to batches
    rows = [reader[i: i + batch_size] for i in range(0, len(reader), batch_size)] 
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=concurrency) as executor:
        executor.map(insert_data, rows)

In [97]:
ratings_query = """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """
load_data('Data/Ratings.csv', ratings_query)

In [98]:
users_query = """
                INSERT INTO users (User_ID, Location, Age)
                VALUES (%s, %s, %s)
                """
load_data('Data/Users.csv', users_query)

In [99]:
books_query = """
            INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
load_data('Data/Books.csv', books_query)

In [100]:
import csv

# Data ingestion for "books"
with open('Data/Books.csv', 'r') as f:
    next(f)  # Skip the header row.
    reader = csv.reader(f)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(insert_row, reader)

# Validation


In [101]:


with open('Data/Ratings.csv', 'r') as f:
    next(f)  # Skip the header row.
    reader = csv.reader(f)
    for row in reader:
        session.execute(
            """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """,
            (row[0], row[1], int(row[2]))
        )

KeyboardInterrupt: 

In [ ]:
ratings_query =             """
            INSERT INTO book_ratings (User_ID, ISBN, Book_Rating)
            VALUES (%s, %s, %s)
            """
load_data('Data/Ratings.csv', ratings_query)

In [ ]:
def load_data(csv_file, insert_query):
        
    def insert_data(rows):
        batch = BatchStatement()
        for row in rows:
            age = float(row[2]) if row[2] else None
            batch.add(SimpleStatement(insert_query), (row[0], row[1], age))
        session.execute(batch)
        print("exec")
    
    with open(csv_file, "r") as f:
        next(f)  # Skip the header row.
        reader = list(csv.reader(f))
    
    # Adjust the number of rows for each batch and number of workers as needed
    num_rows_per_batch = 100
    rows = [reader[i: i + num_rows_per_batch] for i in range(0, len(reader), num_rows_per_batch)] 
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        executor.map(insert_data, rows)

In [ ]:
users_query = """
                INSERT INTO user_data (User_ID, Location, Age)
                VALUES (%s, %s, %s)
                """
load_data('Data/Users.csv', users_query)

In [ ]:
books_query = """
            INSERT INTO books (ISBN, Book_Title, Book_Author, Year_Of_Publication, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
load_data('Data/Books.csv', books_query)

In [ ]:
count_query = "SELECT COUNT(*) FROM user_data"
result = session.execute(count_query)
result.one()

Row(count=278858)

In [ ]:
count_query = "SELECT COUNT(*) FROM books"
result = session.execute(count_query)
result.one()

Row(count=157963)

## Step 3: Creating Tables

In [ ]:
USE university;

In [ ]:
CREATE TABLE students (
  student_id UUID PRIMARY KEY,
  name TEXT,
  email TEXT,
  enrollment_year INT
);

In [ ]:
CREATE TABLE courses (
  course_id UUID PRIMARY KEY,
  course_name TEXT,
  lecturer_id UUID,
  credits INT
);

In [ ]:
CREATE TABLE lecturers (
  lecturer_id UUID PRIMARY KEY,
  name TEXT,
  department TEXT
);

In [ ]:
CREATE TABLE course_enrollments (
  course_id UUID,
  student_id UUID,
  enrollment_date DATE,
  PRIMARY KEY (course_id, student_id)
);

In [ ]:
# To validate you created the tables
cqlsh:university> DESCRIBE tables;

In [ ]:
# The output should be:
course_enrollments  courses  lecturers  students

## Step 4: Inserting data

In [ ]:
# Inserting data into students
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (550e8400-e29b-41d4-a716-446655440000, 'John Doe', 'john.doe@email.com', 2021);
INSERT INTO students (student_id, name, email, enrollment_year) VALUES (652e8500-f39c-42d5-b517-557655450001, 'Jane Smith', 'jane.smith@email.com', 2020);
INSERT INTO students (student_id, name, enrollment_year) VALUES (753e8600-e49d-53e6-c628-668655460002, 'Alice Johnson', 2019);

In [ ]:
# Inserting data into lecturers
INSERT INTO lecturers (lecturer_id, name, department) VALUES (860e9700-a59e-63f7-d739-779655470003, 'Dr. James Watson', 'Biology');
INSERT INTO lecturers (lecturer_id, name, department) VALUES (961e9800-b69f-73c8-e840-889655480004, 'Dr. Emma Wilson', 'Physics');
INSERT INTO lecturers (lecturer_id, name) VALUES (072e9900-d710-83e9-f951-990655490005, 'Prof. Michael Brown');

In [ ]:
# Inserting data into courses
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (183fa000-f821-93a0-b062-a96655650006, 860e9700-a59e-63f7-d739-779655470003, 'Biology 101', 3);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (284fb000-a932-a4b1-c173-b97755660007, 961e9800-b69f-73c8-e840-889655480004, 'Physics 101', 4);
INSERT INTO courses (course_id, lecturer_id, course_name, credits) VALUES (385fc000-d043-b5e2-f284-c98855670008, 072e9900-d710-83e9-f951-990655490005, 'Mathematics 101', 3);

In [ ]:
# Inserting data into course_enrollments
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (183fa000-f821-93a0-b062-a96655650006, 550e8400-e29b-41d4-a716-446655440000, '2023-10-01');
INSERT INTO course_enrollments (course_id, student_id, enrollment_date) VALUES (284fb000-a932-a4b1-c173-b97755660007, 652e8500-f39c-42d5-b517-557655450001, '2023-09-15');
INSERT INTO course_enrollments (course_id, student_id) VALUES (385fc000-d043-b5e2-f284-c98855670008, 753e8600-e49d-53e6-c628-668655460002);

## Step 5: Querying the data

In [ ]:
# Get all students
SELECT * FROM STUDENTS

>>
 student_id                           | email                | enrollment_year | name
--------------------------------------+----------------------+-----------------+---------------
 753e8600-e49d-53e6-c628-668655460002 |                 null |            2019 | Alice Johnson
 652e8500-f39c-42d5-b517-557655450001 | jane.smith@email.com |            2020 |    Jane Smith
 550e8400-e29b-41d4-a716-446655440000 |   john.doe@email.com |            2021 |      John Doe
 (3 rows)

In [ ]:
# Get the list of students who enrolled after 2020 along with their email and the courses they enrolled in
SELECT student_id, name, email
FROM students
WHERE enrollment_year > 2020 ALLOW FILTERING;

>>
 student_id                           | name     | email
--------------------------------------+----------+--------------------
 550e8400-e29b-41d4-a716-446655440000 | John Doe | john.doe@email.com

(1 rows)

In [ ]:
# Find all courses and count of students enrolled in them
SELECT course_id, COUNT(student_id) as student_count
FROM course_enrollments
GROUP BY course_id;

>>
 course_id                            | student_count
--------------------------------------+---------------
 183fa000-f821-93a0-b062-a96655650006 |             1
 385fc000-d043-b5e2-f284-c98855670008 |             1
 284fb000-a932-a4b1-c173-b97755660007 |             1

(3 rows)